In [1]:

from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    PreTrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)

In [9]:
# tokenizer = AutoTokenizer.from_pretrained("/data/z_workspace/knowledge/model/ChatGLM3-6b", trust_remote_code=True)
# model = AutoModel.from_pretrained("/data/z_workspace/knowledge/finetune_demo/ptuning_output/checkpoint-3000", trust_remote_code=True, device='cuda:1')
# model = model.eval()
def load_model_and_tokenizer(model_dir, trust_remote_code: bool = True):
     model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=trust_remote_code, device_map='cuda:0')
     tokenizer = AutoTokenizer.from_pretrained(
        '/data/z_workspace/knowledge/model/ChatGLM3-6b', trust_remote_code=trust_remote_code
    )
     return model, tokenizer
model,tokenizer = load_model_and_tokenizer('/data/z_workspace/knowledge/finetune_demo/lora_output/checkpoint-3000')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
prompt = f"""提示:目前，我的图数据库中有8类实体：疾病、药品、药品商、疾病症状、食物、检查项目、治疗方法、科目。在这里面，“疾病”这个实体有7个属性：名称、治愈概率、治疗周期、疾病易感人群、疾病病因、疾病简介、预防措施；其余实体均只有名称这个属性。在实体之间，具有8个关系：疾病使用药品、治疗的方法、疾病宜吃食物、疾病忌吃食物、疾病的症状、疾病所需检查、疾病所属科目、疾病并发疾病。其中“疾病使用药品”是“疾病”和“药品”之间的关系，“治疗的方法”是“疾病”和“治疗方法”之间的关系，“疾病宜吃食物”是“疾病”和“食物”之间的关系，“疾病忌吃食物”是“疾病”和“食物”之间的关系，“疾病的症状”是“疾病”和“疾病症状”之间的关系，“疾病所需检查”是“疾病”和“检查项目”之间的关系，“疾病所属科目”是“疾病”和“科目”之间的关系，“疾病并发疾病”是“疾病”和“疾病”之间的关系。
    
现在，你是一个机器人医生，用户对你输入问题，你需要精准的理解问题的内容，根据其含义构建Neo4j数据库的查询语句，多条查询语句请用逗号分割。
    
请注意，你的查询语句只限于两个操作：1.查询疾病的属性，你需要提供疾病具体名称和属性名。2.查询疾病与其他实体之间的关系，你需要提供疾病具体名称、关系名称、实体类别名。为了简化输出，你输出查询语句格式的应为：对于类型1的查询，输出“1 疾病名称 属性”；对于类型2的查询，输出“2 疾病名称 关系 实体类别”。如果有多个查询，每个查询之间用中文逗号隔开。
    
用户问题：{'得了急性阑尾炎该怎么办？'}
查询语句:
解释:
    """
response, _ = model.chat(tokenizer, prompt)
print(response)

查询语句: 1 急性阑尾炎 疾病简介, 2 急性阑尾炎 治疗的方法 治疗方法, 2 急性阑尾炎 疾病使用药品 药品

解释: 用户询问的是急性阑尾炎的处理方法，因此首先需要查询急性阑尾炎的疾病简介，以便用户了解疾病的基本情况。接着，用户最关心的是治疗方法，所以需要提供急性阑尾炎的治疗方法。此外，用户可能还需要了解有哪些药品可以用于治疗，因此也需查询“疾病使用药品”关系。
